In [ ]:
def conv_size_out(size_in, kern, stride):
    pad = 0
    dilation = 1
    return (size_in + 2*pad - dilation*(kern - 1) - 1) // stride + 1

def avg_size_out(size_in, kern, stride):
    pad = 0
    return (size_in + 2*pad - kern) // stride + 1

def max_size_out(size_in, kern, stride):
    pad = 0
    dilation = 1
    return (size_in + 2*pad - dilation*(kern - 1) - 1) // stride + 1

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.model = nn.Sequential(            
#             nn.MaxPool2d(4,2),
            nn.Flatten(),
#             nn.Linear(in_features=225, out_features=10)
        )

    def forward(self, x):
        logits = self.model(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs

In [ ]:
x = torch.rand(100,1,32,32)
y = torch.randint(low=0,high=9,size=(100,))

In [ ]:
train_dataset = torch.utils.data.TensorDataset(x, y)

train_loader = torch.utils.data.DataLoader(
        dataset = train_dataset,
        batch_size = 32,
        shuffle = True
    )

In [ ]:
def profile(model, train_loader):    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
    
    def train(epochs):
        total_step = len(train_loader)
        print(total_step)

        for epoch in range(epochs):
            for i, (images, labels) in enumerate(train_loader):
                # Forward pass
                outputs = model(images)            
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if (i+1) % 10 == 0:
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {}' 
                           .format(epoch+1, epochs, i+1, total_step, loss))
    
    EPOCHS = 1
    with torch.autograd.profiler.profile() as prof:
        train(EPOCHS)
        
    return prof.key_averages()

In [ ]:
ls = profile(Model().model,train_loader)

In [ ]:
print(ls)

In [ ]:
for x in ls:
    print(x.key)
    print()

In [ ]:
model = Model().model

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
for i, (images, labels) in enumerate(train_loader):
    # Forward pass
    with torch.autograd.profiler.profile() as prof:
        outputs = model(images)
    break

    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    break



In [ ]:
print(prof.key_averages())

In [ ]:
for k in prof.key_averages():
    print(k.key)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers  
from tensorflow.keras.layers import Dense, Flatten

from numpy.random import RandomState as R

seed = 42

def give(dim, n, channels):
    ds_size = 1024
    out_size = 10
    if dim == 1:
        x = R(seed).random((ds_size, n, channels))
        x = x.reshape(x.shape[0], n, channels)
    else:
        x = R(seed).random((ds_size, n, n, channels))
        x = x.reshape(x.shape[0], n, n, channels)
    
    y = R(seed).randint(0,out_size,ds_size)
    y = tf.keras.utils.to_categorical(y, out_size)
    
    return x,y

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.01)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

def profile(model, x, y, batch, epochs):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.batch(batch)
    
    EPOCHS = epochs
    prof_file = 'out_tflow.csv'
    logdir = 'logs'

    with tf.profiler.experimental.Profile(logdir):
        model.fit(dataset, epochs = EPOCHS)
        pass

    return prof_file
    
class Model:
    def create(self):
        model = Sequential()
        model.add( 
            Conv2D(filters = 3, kernel_size = 4, strides = 2)
        )
        model.add( Flatten(name='FLATTEN') )
        model.add( Dense(units = 10, name='FINAL_DENSE') )
        model.compile(loss = loss, optimizer = opt, metrics=['accuracy'])
        self.model = model

Model = Model()
Model.create()

x,y = give(2, 32, 1)

In [ ]:
prof = profile(Model.model, x, y, 32, 1)

In [ ]:
import data_parser as dp

In [ ]:
the_dir = ['intermediate_results', 'ptorch']

In [ ]:
import os

In [ ]:
dct = dp.load(os.path.join(*the_dir, 'dense.torch'))

In [ ]:
for x,y in dct.items():
    break

In [ ]:
y.sort_values(by='CPU total (us)')

In [1]:
import data_parser as dp

In [2]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
dct = dp.load('profiling_results/vms_1/__avg1d.tflow')

In [7]:
for k in dct.values():
    break

In [9]:
k.sort_values(by='Operation', ascending=False)

,Type,Operation,#Occurrences,Avg. self-time (us),Total self-time (us),Avg. time (us),Total time (us)
59,StridedSlice,strided_slice,20,24.314000,486.280,24.314000,486.280
55,ReadVariableOp,sequential/FINAL_DENSE/MatMul/ReadVariableOp,20,5.145550,102.911,5.145550,102.911
54,ReadVariableOp,sequential/FINAL_DENSE/BiasAdd/ReadVariableOp,20,2.920050,58.401,2.920050,58.401
70,_FusedMatMul,sequential/FINAL_DENSE/BiasAdd,20,2143.519500,42870.390,2143.519500,42870.390
19,Cast,sequential/Cast,20,79.074150,1581.483,79.074150,1581.483
12,AvgPool,sequential/AVG1D/AvgPool,20,374.683400,7493.668,374.683400,7493.668
37,MatMul,gradient_tape/sequential/FINAL_DENSE/MatMul,20,49.992000,999.840,49.992000,999.840
14,BiasAddGrad,gradient_tape/sequential/FINAL_DENSE/BiasAdd/BiasAddGrad,20,11.009600,220.192,11.009600,220.192
28,DivNoNan,gradient_tape/categorical_crossentropy/weighted_loss/value/div_no_nan,20,2.110850,42.217,2.110850,42.217
67,Tile,gradient_tape/categorical_crossentropy/weighted_loss/Tile_1,20,6.580700,131.614,6.580700,131.614


In [5]:
t=k[['Type', 'Total self-time (us)']].groupby(by='Type').sum()

In [6]:
t.sort_values(by='Total self-time (us)', ascending=False)

,Total self-time (us)
Type,
IDLE,1134280.877
_FusedMatMul,42870.390
Dataset,13923.100
OptimizeDataset,12698.664
AvgPool,7493.668
SoftmaxCrossEntropyWithLogits,2523.884
Cast,1778.266
ArgMax,1121.439
DeleteIterator,1079.745
